# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
import datetime

from PIL import Image

from skimage import data
from skimage.transform import resize
# from scipy.misc import imread, imresize # read image from disc -- bring image to common shape

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('../DATA Human Gestures/train.csv').readlines())
val_doc = np.random.permutation(open('../DATA Human Gestures/val.csv').readlines())
# experimentation with batch size
batch_size = 256 # memory efficient and maximum dat to fee din network

## Generator

* The overall structure of the generator has been given. 
* An interesting thing to note here is the use of the infinite while loop. 
    * It is there in place so that the generator is always ready to yield a batch once next() is called once it is called at the start of training. 
    * Even after one pass over the data is completed (after the for loop is completed and the batch for the remainder datapoints is yielded), upon the subsequent next() call (at the start of the next epoch), the processing starts from the command 't=np.random.permuatation(folder_list)'. 
    * In this way, the generator requires very less memory while training.


* In the generator, 
    - Preprocess the images as you have images of 2 different dimensions,
    - as well as create a batch of video frames.

* Experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [15]:
def generator(source_path, folder_list):
    '''
    source_path: root directory
    folder_list: document
    batch_size: data size in each batch
    '''
    
    global batch_size
    
    print( 'Source path = ', source_path, ' ; batch size =', batch_size)
    
    # experiment with creating a list of image numbers you want to use for a particular video, 
    # out of all frame images in each folder -- improve performance
    # list of index of images
    img_idx = [1,2,5,4,8,7]
    
    # continuos flow of data -- for several epochs
    while True:
        
        # randomly select from sequence
        t = np.random.permutation(folder_list)
        
        # 1 sequence = 1 video = 30 image collection
        # calculate the number of batches to call in each iterations -- (number of video frames)/(batch size) -- 30/batch_size
        num_batches = 6
        
        # we iterate over the number of batches
        for batch in range(num_batches):     
            # creating 0 tensors -- empty containers -- load images
            # x is the number of images you use for each video = img_idx
            # (y,z) is the final size of the input images and  = height,width
            # 3 is the number of channels RGB
            batch_data = np.zeros((batch_size,x,y,z,3))
            # batch_labels is the one hot ecnoding representation of the output -- 5 classes
            batch_labels = np.zeros((batch_size,5)) 
            
            
            # iterate over the batch_size
            for folder in range(batch_size): 
                # read all the images in the video image sequence folder
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                
                # Iterate iver the frames/images of a folder to read them in
                for idx,item in enumerate(img_idx): 
                    # selectively take images
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # crop the images and resize them. Note that the images are of 2 different shape 
                    # and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    # normalise and feed in the image
                    batch_data[folder,idx,:,:,0] =   # red
                    batch_data[folder,idx,:,:,1] =   # green
                    batch_data[folder,idx,:,:,2] =   # blue
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
            #you yield the batch_data and the batch_labels, remember what does yield do
            yield batch_data, batch_labels 
            
            # write the code for the remaining data points which are left after full batches


SyntaxError: invalid syntax (3362442636.py, line 47)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [19]:
# current datetime
curr_dt_time = datetime.datetime.now()

# paths
train_path = '../DATA Human Gestures/train'
val_path = '../DATA Human Gestures/val'

# all training sequences
num_train_sequences = len(train_doc)
print('training sequences =', num_train_sequences)

# all validation sequences
num_val_sequences = len(val_doc)
print('validation sequences =', num_val_sequences)

# choose the number of epochs
num_epochs = 10
print ('epochs =', num_epochs)

training sequences = 663
validation sequences = 100
epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. 
* Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. 
* Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

 - Start from small model and small amount of data, extend model layers and data size.
 - Use high batch size and gradually reduce batch size

In [ ]:
#write your model here

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [1]:
# write your optimizer, loss fnction
optimiser = 'SGD'
loss_function = 'categorical_crossentropy'
metrics_list = ['categorical_accuracy']
model.compile(
    # learning optimiser
    optimizer=optimiser, 
    # loss function
    loss=loss_function, 
    # metrics list
    metrics=metrics_list,
)

print (model.summary())

NameError: name 'model' is not defined

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
# training batch
train_generator = generator(train_path, train_doc)
# validaitn batch
val_generator = generator(val_path, val_doc)

In [ ]:
# save model -- learning weights with every epoch -- current time
model_name = 'model_gesture' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
    
# save model detailsin filename at every epoch in disc memory-- epoch,loss,accuracy
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

# automatically save model after every epoch
checkpoint = ModelCheckpoint(
    filepath, monitor='val_loss', verbose=1, save_best_only=False, 
    save_weights_only=False, mode='auto', period=1
)

# write the REducelronplateau code here
'''
1. if validation loss not changing, 
2. we have hit a plateau and not changing.
3. reduce learning rate to move towards minima and doesn't change much

'''
LR = 
# pass as a lit to fit.generator, check at every epoch, if validation loss is not decreasing, reduce learning rate
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [2]:
# number of next calls fit_generator will make
'''
num_train_sequences,num_val_sequences = number of videos in training & validation data
if training_sequence is divisible by batch size, take the division else floor-diviison+1
'''

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

NameError: name 'num_train_sequences' is not defined

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit_generator(
    # pass generator object -- calls training and validation data
    train_generator, 
    # number of batch calls per epoch
    steps_per_epoch=steps_per_epoch, 
    # number of epoxhs
    epochs=num_epochs, 
    # book keeping for model performance with every epoch
    verbose=1, 
    # check at avery epochs
    callbacks=callbacks_list, 
    # test model til the epoch with validatindata
    validation_data=val_generator, 
    # number of calls per epoch for validatin data
    validation_steps=validation_steps, 
    # if unbalanced data is present
    class_weight=None, 
    # fetching and computing happens at same time -- parallel computing -- 
    '''cpu fetch data -- gpu computes model on previous fetched data -- fit.generator'''
    # 
    workers=1, 
    # initialize 
    initial_epoch=0
)